In [3]:
import numpy as np
import pandas as pd
import glob as gb 
import os

### Состояние склада на каждый день

Создание имени файла

In [1]:
def create_filenames(letter, number):
    
    c = chr(ord('0') + number)
    str_i = 'MS-' + letter + c + '-inventory.csv'
    str_sl = 'MS-' + letter + number + '-sell.csv'
    str_spl = 'MS-' + letter + number + '-supply.csv'
    str_d = 'MS-' + letter + number + '-daily.csv'
    return str_i, str_sl, str_spl, str_d

Количество магазинов в штате

In [2]:
def num_shop(state):
    
    if state == 'b' or state == 'm':
        max = 2
    elif state == 's':
        max = 5
    return max

In [ ]:
def daily_stor(state):
    
    max = num_shop(state)
    files = {}
    path = {}
    answer = {}
    shop = 0
    while shop < max:
        c = chr(ord('0') + shop + 1)
        path[shop] = gb.glob(os.getcwd() + '/input/*'+ state + c + '*.csv')
        for j in range(0, len(path[shop])):
            files[shop][j]= pd.read_csv(path[shop][j])
        shop = shop + 1
    INV = 0
    SELL = 1
    SPL = 2
    DAILY = 3
    
    shop = 0
    while shop < max:
        files[shop][SELL]['sku_num'] = files[shop][SELL]['sku_num'].apply(lambda x: 0 if x.find('ap') == 6 else 1)
        daily = pd.crosstab(files[shop][SELL]['date'], files[shop][SELL]['sku_num'], margins = True)
        daily_new = pd.DataFrame(columns=['date', 'apple_sell', 'pen_sell'])
        daily_new['date'] = daily.index
        daily_new['apple_sell'] = daily[0].values
        daily_new['pen_sell'] = daily[1].values
        
        files[shop][SPL]['date'] = pd.to_datetime(files[shop][SPL]['date'], format='%Y-%m-%d')
        files[shop][SPL].index = files[shop][SPL]['date']
        
        l = files[shop][SPL].index.size
        files[shop][SPL]['apple'] = files[shop][SPL]['apple'].rolling(str(l) + 'D').sum()
        files[shop][SPL]['pen'] = files[shop][SPL]['pen'].rolling(str(l) + 'D').sum()
        
       # files[shop][SELL]['date'] = pd.to_datetime(files[shop][SELL]['date'], format='%Y-%m-%d')
       # files[shop][SELL].index = files[shop][SELL]['date']
        
        more_inf = pd.merge(daily_new, files[shop][SPL], how='outer', on='date')
        
        files[shop][SELL]['date'] = pd.to_datetime(files[shop][SELL]['date'], format='%Y-%m-%d')
        more_inf.index = files[shop][SELL]['date'].drop_duplicates()
        more_inf = more_inf.drop(['date'], axis=1)
        
        l = more_inf.index.size
        more_inf['apple_sum'] = more_inf['apple_sell'].rolling(str(l) + 'D').sum()
        more_inf['pen_sum'] = more_inf['pen_sell'].rolling(str(l) + 'D').sum()
        
        more_inf['apple'] = more_inf['apple'] - more_inf['apple_sum']
        more_inf['pen'] = more_inf['pen'] - more_inf['pen_sum']
        
        more_inf = more_inf.drop(['pen_sell', 'apple_sell', 'apple_sum', 'pen_sum'], axis=1)

        name = create_filenames(state, shop + 1)[DAILY]
        more_inf.to_csv('answer/' + name, index=False)
        answer[shop] = more_inf
        shop = shop + 1
        
    return answer
    

In [5]:
def main():
    
    daily_b = daily_stor('b')
    print(daily_b)
    daily_m = daily_stor('m')
    print(daily_m)
    daily_s = daily_stor('s')
    print(daily_s)
    
main() 
    

['/home/varvara/6_sem_prac/task_3/input/MS-s5-sell.csv', '/home/varvara/6_sem_prac/task_3/input/MS-m2-sell.csv', '/home/varvara/6_sem_prac/task_3/input/MS-b2-sell.csv', '/home/varvara/6_sem_prac/task_3/input/MS-s4-sell.csv', '/home/varvara/6_sem_prac/task_3/input/MS-s2-sell.csv', '/home/varvara/6_sem_prac/task_3/input/MS-s1-sell.csv', '/home/varvara/6_sem_prac/task_3/input/MS-s3-sell.csv', '/home/varvara/6_sem_prac/task_3/input/MS-b1-sell.csv', '/home/varvara/6_sem_prac/task_3/input/MS-m1-sell.csv']
